In [32]:
%load_ext autoreload
%autoreload 2

import logging
import os
from os.path import join as pj

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
from sae_lens import SAE, ActivationsStore
from transformer_lens import HookedTransformer

from sae_cooccurrence.normalised_cooc_functions import (
    create_results_dir,
)
from sae_cooccurrence.pca import (
    analyze_specific_points_from_sparse_matrix_path,
    generate_data,
    load_data_from_pickle,
    plot_doubly_clustered_activation_heatmap,
    plot_pca_explanation_and_save,
    plot_pca_feature_strength,
    plot_pca_single_feature_strength,
    plot_pca_with_active_features,
    plot_pca_with_top_feature,
    plot_simple_scatter,
    plot_subgraph_from_sparse_matrix_path,
    plot_token_pca_and_save,
    save_data_to_pickle,
)
from sae_cooccurrence.utils.saving_loading import load_npz_files, set_device
from sae_cooccurrence.utils.set_paths import get_git_root

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def setup_logging(log_path):
    logging.basicConfig(
        filename=log_path,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )

In [3]:
# Config -------------
torch.set_grad_enabled(False)
device = set_device()
git_root = get_git_root()

Using MPS


In [4]:
save_figs = True


model_name = "gpt2-small"
sae_release_short = "res-jb"
sae_id = "blocks.0.hook_resid_pre"
n_batches_reconstruction = 1000


activation_threshold = 1.5
subgraph_id = 2644
n_batches_generation = 1000

In [5]:
fs_splitting_cluster = subgraph_id
pca_prefix = "pca"

np.random.seed(1234)


# n_batches_reconstruction = config['pca']['n_batches_reconstruction']


# Load model
model = HookedTransformer.from_pretrained(model_name, device=device)

# Process the specific subgraph
sae_id_neat = sae_id.replace(".", "_")
results_dir = create_results_dir(
    model_name, sae_release_short, sae_id_neat, n_batches_generation
)
results_path = pj(git_root, results_dir)
activation_threshold_safe = str(activation_threshold).replace(".", "_")

figures_path = pj(git_root, f"figures/{model_name}/{sae_release_short}/{sae_id_neat}")
pca_dir = f"{pca_prefix}_{activation_threshold_safe}_subgraph_{subgraph_id}"
pca_path = pj(figures_path, pca_dir)
if not os.path.exists(pca_path):
    os.makedirs(pca_path)
pickle_file = pj(pca_path, f"pca_data_subgraph_{subgraph_id}.pkl")

# Set up logging
log_path = pj(pca_path, "pca_analysis.log")
setup_logging(log_path)

# Log all settings
logging.info("Script started")
logging.info("Settings:")
logging.info(f"  save_figs: {save_figs}")
logging.info(f"  git_root: {git_root}")
logging.info(f"  sae_id: {sae_id}")
logging.info(f"  activation_threshold: {activation_threshold}")
logging.info(f"  subgraph_id: {subgraph_id}")
logging.info(f"  fs_splitting_cluster: {fs_splitting_cluster}")
logging.info(f"  pca_prefix: {pca_prefix}")
logging.info(f"  model_name: {model_name}")
logging.info(f"  sae_release_short: {sae_release_short}")
logging.info(f"  n_batches_reconstruction: {n_batches_reconstruction}")
logging.info(f"  device: {device}")
logging.info(f"  results_path: {results_path}")
logging.info(f"  pca_path: {pca_path}")

Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
node_df = pd.read_csv(
    pj(results_path, f"dataframes/node_info_df_{activation_threshold_safe}.csv")
)
logging.info(
    f"Loaded node_df from {pj(results_path, f'dataframes/node_info_df_{activation_threshold_safe}.csv')}"
)

overall_feature_activations = load_npz_files(
    results_path, "feature_acts_cooc_activations"
).get(activation_threshold)

# with open(pj(results_path, f"subgraph_objects/activation_{activation_threshold_safe}/subgraph_{subgraph_id}.pkl"), 'rb') as f:
#     subgraph = pickle.load(f)


# Filter for the specific subgraph
fs_splitting_nodes = node_df.query("subgraph_id == @subgraph_id")["node_id"].tolist()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading npz files:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
regen_data = False
if not regen_data:
    raise ValueError("Are you sure you don't want to use existing data?")

ValueError: Are you sure you don't want to use existing data?

In [9]:
# parser = argparse.ArgumentParser(description="PCA analysis script")
# parser.add_argument('--save_pickle', action='store_true', help='Save generated data to pickle')
# parser.add_argument('--load_pickle', action='store_true', help='Load data from pickle instead of regenerating')
# args = parser.parse_args()


if not regen_data and os.path.exists(pickle_file):
    data = load_data_from_pickle(pickle_file)
    results = data["results"]
    pca_df = data["pca_df"]
    pca = data["pca"]
    pca_decoder = data["pca_decoder"]
    pca_decoder_df = data["pca_decoder_df"]
else:
    # Load SAE and set up activation store
    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=f"{model_name}-{sae_release_short}", sae_id=sae_id, device=device
    )
    sae.fold_W_dec_norm()

    activation_store = ActivationsStore.from_sae(
        model=model,
        sae=sae,
        streaming=True,
        store_batch_size_prompts=8,
        train_batch_size_tokens=4096,
        n_batches_in_buffer=32,
        device=device,
    )

    data = generate_data(
        model,
        sae,
        activation_store,
        fs_splitting_nodes,
        n_batches_reconstruction,
        device=device,
    )

    if regen_data:
        save_data_to_pickle(data, pickle_file)

    results = data["results"]
    pca_df = data["pca_df"]
    pca = data["pca"]
    pca_decoder = data["pca_decoder"]
    pca_decoder_df = data["pca_decoder_df"]

In [11]:
plot_token_pca_and_save(pca_df, pca_path, subgraph_id, color_by="token", save=save_figs)

plot_pca_explanation_and_save(pca, pca_path, subgraph_id, save=save_figs)

plot_simple_scatter(results, pca_path, subgraph_id, fs_splitting_nodes, save=save_figs)

# pca_decoder, pca_decoder_df = calculate_pca_decoder(sae, fs_splitting_nodes)

# Save pca_decoder_df as CSV
# pca_decoder_df_filename = f"pca_decoder_df_subgraph_{subgraph_id}.csv"
# pca_decoder_df.to_csv(pj(pca_path, pca_decoder_df_filename), index=False)

# create_pca_plots_decoder(pca_decoder_df, subgraph_id, pca_path, save=save_figs)

print(f"Processing completed for subgraph ID {subgraph_id}")

Processing completed for subgraph ID 2644


In [12]:
plot_pca_with_top_feature(
    pca_df, results, fs_splitting_nodes, fs_splitting_cluster, pca_path, save=save_figs
)

In [13]:
plot_pca_feature_strength(
    pca_df,
    results,
    fs_splitting_nodes,
    fs_splitting_cluster,
    pca_path,
    pc_x="PC1",
    pc_y="PC2",
    save=save_figs,
)
plot_pca_feature_strength(
    pca_df,
    results,
    fs_splitting_nodes,
    fs_splitting_cluster,
    pca_path,
    pc_x="PC1",
    pc_y="PC3",
    save=save_figs,
)
plot_pca_feature_strength(
    pca_df,
    results,
    fs_splitting_nodes,
    fs_splitting_cluster,
    pca_path,
    pc_x="PC2",
    pc_y="PC3",
    save=save_figs,
)

In [14]:
plot_pca_feature_strength(
    pca_df,
    results,
    fs_splitting_nodes,
    fs_splitting_cluster,
    pca_path,
    pc_x="PC2",
    pc_y="PC3",
    save=save_figs,
)

In [15]:
fs_splitting_nodes

[17827, 3877, 15526, 2662, 17351, 6890, 4622, 15727, 18994, 10676, 8789, 22458]

In [18]:
plot_pca_single_feature_strength(
    pca_df=pca_df,
    results=results,
    feature_index=10676,
    fs_splitting_cluster=fs_splitting_cluster,
    pca_path=pca_path,
    pc_x="PC2",
    pc_y="PC3",
    # activation_threshold=activation_threshold,
    save=save_figs,
)

Plots saved as PNG: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_10676_PC2_vs_PC3.png
Plots saved as SVG: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_10676_PC2_vs_PC3.svg
Plots saved as HTML: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_10676_PC2_vs_PC3.html


In [19]:
plot_pca_single_feature_strength(
    pca_df,
    results,
    feature_index=3877,
    fs_splitting_cluster=fs_splitting_cluster,
    pca_path=pca_path,
    pc_x="PC2",
    pc_y="PC3",
    # activation_threshold=activation_threshold,
    save=save_figs,
)

Plots saved as PNG: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_3877_PC2_vs_PC3.png
Plots saved as SVG: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_3877_PC2_vs_PC3.svg
Plots saved as HTML: /Users/matthew/Documents/Github/sae_cooccurrence/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_3877_PC2_vs_PC3.html


In [26]:
# plot_pca_single_feature_strength(
#     pca_df,
#     results,
#     8838,
#     fs_splitting_cluster,
#     pca_path,
#     pc_x="PC2",
#     pc_y="PC3",
#     activation_threshold=activation_threshold,
#     save=save_figs,
# )

Plots saved as PNG: /Users/matthew/Documents/Github/PIBBSS/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_8838_PC2_vs_PC3.png
Plots saved as SVG: /Users/matthew/Documents/Github/PIBBSS/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_8838_PC2_vs_PC3.svg
Plots saved as HTML: /Users/matthew/Documents/Github/PIBBSS/figures/gpt2-small/res-jb/blocks_0_hook_resid_pre/pca_1_5_subgraph_2644/pca_plot_graph_2644_feature_8838_PC2_vs_PC3.html


In [20]:
plot_pca_with_active_features(
    pca_df,
    results,
    fs_splitting_nodes,
    fs_splitting_cluster,
    pca_path,
    activation_threshold=activation_threshold,
    save=save_figs,
)

In [21]:
plot_doubly_clustered_activation_heatmap(
    results,
    fs_splitting_nodes,
    pca_df,
    pca_path,
    fs_splitting_cluster,
    max_examples=1000,
    save=save_figs,
)

In [22]:
# plot_feature_activations_combined(
#     get_point_result(results, 2),
#     fs_splitting_nodes,
#     fs_splitting_cluster,
#     activation_threshold,
#     node_df,
#     results_path,
#     pca_path,
#     save_figs=True,
# )

In [23]:
# plot_feature_activations(
#     get_point_result(results, 2),
#     fs_splitting_nodes,
#     fs_splitting_cluster,
#     activation_threshold,
#     node_df,
#     results_path,
#     save_figs=False,
#     pca_path=pca_path,
# )


FileNotFoundError: Subgraph file not found: /Users/matthew/Documents/Github/sae_cooccurrence/results/gpt2-small/res-jb/blocks_0_hook_resid_pre/n_batches_1000/subgraph_objects/activation_1_5/subgraph_2644.pkl

In [31]:
# Usage example:
# pca_df, _ = perform_pca_on_results(results)
# analyze_representative_points(
#     results=results,
#     fs_splitting_nodes=fs_splitting_nodes,
#     fs_splitting_cluster=fs_splitting_cluster,
#     activation_threshold=activation_threshold,
#     node_df=node_df,
#     results_path=results_path,
#     pca_df=pca_df,
#     save_figs=True,
#     pca_path=pca_path,
# )


Analyzing representative point 1:


/Users/matthew/Documents/Github/PIBBSS/PIBBSS/graph_generation.py:586: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.

/Users/matthew/Documents/Github/PIBBSS/PIBBSS/graph_generation.py:586: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Number of non-zero features: 10
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6728
Mean activation of non-zero non-feature splitting nodes: 0.4737
Median activation of non-zero feature splitting nodes: 2.6728
Median activation of non-zero non-feature splitting nodes: 0.1765
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 1
Sum of activation strengths for splitting features: 2.6728
Sum of activation strengths for non-splitting features: 4.2630

Analyzing representative point 2:


/Users/matthew/Documents/Github/PIBBSS/PIBBSS/graph_generation.py:586: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Number of non-zero features: 6
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6709
Mean activation of non-zero non-feature splitting nodes: 0.1844
Median activation of non-zero feature splitting nodes: 2.6709
Median activation of non-zero non-feature splitting nodes: 0.1149
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 0
Sum of activation strengths for splitting features: 2.6709
Sum of activation strengths for non-splitting features: 0.9222

Analyzing representative point 3:


...

Number of non-zero features: 14
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6684
Mean activation of non-zero non-feature splitting nodes: 0.1870
Median activation of non-zero feature splitting nodes: 2.6684
Median activation of non-zero non-feature splitting nodes: 0.0903
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 0
Sum of activation strengths for splitting features: 2.6684
Sum of activation strengths for non-splitting features: 2.4304

Analyzing representative point 4:


/Users/matthew/Documents/Github/PIBBSS/PIBBSS/graph_generation.py:586: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Number of non-zero features: 13
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6043
Mean activation of non-zero non-feature splitting nodes: 0.1746
Median activation of non-zero feature splitting nodes: 2.6043
Median activation of non-zero non-feature splitting nodes: 0.1128
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 0
Sum of activation strengths for splitting features: 2.6043
Sum of activation strengths for non-splitting features: 2.0955

Analyzing representative point 5:


/Users/matthew/Documents/Github/PIBBSS/PIBBSS/graph_generation.py:586: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Number of non-zero features: 13
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6043
Mean activation of non-zero non-feature splitting nodes: 0.1746
Median activation of non-zero feature splitting nodes: 2.6043
Median activation of non-zero non-feature splitting nodes: 0.1128
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 0
Sum of activation strengths for splitting features: 2.6043
Sum of activation strengths for non-splitting features: 2.0955


In [32]:
# analyze_representative_points_comp(
#     results,
#     fs_splitting_nodes,
#     activation_threshold,
#     node_df,
#     pca_df,
#     save_figs=True,
#     pca_path=pca_path,
# )


Statistics for representative point 1:
Number of non-zero features: 10
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6728
Mean activation of non-zero non-feature splitting nodes: 0.4737
Median activation of non-zero feature splitting nodes: 2.6728
Median activation of non-zero non-feature splitting nodes: 0.1765
Number of splitting features active above threshold: 1
Number of non-splitting features active above threshold: 1
Sum of activation strengths for splitting features: 2.6728
Sum of activation strengths for non-splitting features: 4.2630

Statistics for representative point 2:
Number of non-zero features: 6
Number of non-zero feature splitting nodes: 1
Total number of feature splitting nodes: 12
Mean activation of non-zero feature splitting nodes: 2.6709
Mean activation of non-zero non-feature splitting nodes: 0.1844
Median activation of non-zero feature splitting nodes: 2.6709
Med

In [33]:
# After creating the PCA plot and identifying interesting points
interesting_point_ids = [54, 357, 178, 930, 1001]  # Replace with actual IDs of interest


analyze_specific_points_from_sparse_matrix_path(
    results,
    results_path,
    fs_splitting_nodes,
    fs_splitting_cluster,
    activation_threshold,
    node_df,
    pca_df,
    interesting_point_ids,
    save_figs=True,
    pca_path=pca_path,
)


Analyzing point with ID 54:


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Analyzing point with ID 357:


/Users/matthew/Documents/Github/sae_cooccurrence/sae_cooccurrence/pca.py:3539: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Subgraph figure is None. Likely no latents are within a subgraph.

Analyzing point with ID 178:


/Users/matthew/Documents/Github/sae_cooccurrence/sae_cooccurrence/pca.py:3539: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Subgraph figure is None. Likely no latents are within a subgraph.

Analyzing point with ID 930:



Analyzing point with ID 1001:


In [25]:
# analyze_user_specified_points_comp(
#     results,
#     fs_splitting_nodes,
#     activation_threshold,
#     node_df,
#     pca_df,
#     interesting_point_ids,
#     save_figs=True,
#     pca_path=pca_path,
# )

In [26]:
# analyze_user_specified_points_comp_subgraph(
#     results,
#     fs_splitting_nodes,
#     fs_splitting_cluster,
#     activation_threshold,
#     node_df,
#     pca_df,
#     interesting_point_ids,
#     results_path,
#     save_figs=True,
#     pca_path=pca_path,
# )

In [27]:
def plot_pca_months(
    pca_df, pca_path, fs_splitting_cluster, plot_inner=False, save_figs=False
):
    # Define colors for each day and gray for others
    if not plot_inner:
        color_map = {
            "January": "#FF9999",
            "February": "#66B2FF",
            "March": "#99FF99",
            "April": "#FFCC99",
            "May": "#FF99FF",
            "June": "#99FFFF",
            "July": "#FFFF99",
            "August": "#FF9999",
            "September": "#66B2FF",
            "October": "#99FF99",
            "November": "#FFCC99",
            "December": "#FF99FF",
            "Other": "#CCCCCC",
        }
    else:
        color_map = {
            "Jan": "#FF9999",
            "Feb": "#66B2FF",
            "Mar": "#99FF99",
            "Apr": "#FFCC99",
            "May": "#FF99FF",
            "Jun": "#99FFFF",
            "Jul": "#FFFF99",
            "Aug": "#FF9999",
            "Sep": "#66B2FF",
            "Oct": "#99FF99",
            "Nov": "#FFCC99",
            "Dec": "#FF99FF",
            "Other": "#CCCCCC",
        }

    # Function to determine color and marker shape
    def get_color_and_marker(token):
        token_lower = token.lower()
        for day in color_map.keys():
            if day.lower() in token_lower:
                return color_map[day], "cross" if " " in token else "circle"
        return color_map["Other"], "circle"

    # Apply the function to get colors and markers
    pca_df["color"], pca_df["marker"] = zip(
        *pca_df["tokens"].apply(get_color_and_marker)
    )

    # Create the plot
    fig = go.Figure()

    # Add traces for colors (days)
    for day in list(color_map.keys()):
        df_day = pca_df[pca_df["color"] == color_map[day]]
        fig.add_trace(
            go.Scatter(
                x=df_day["PC2"],
                y=df_day["PC3"],
                mode="markers",
                marker=dict(color=color_map[day], size=8),
                name=day,
                legendgroup="days",
                legendgrouptitle_text="Day of Week",
                text=[
                    f"Token: {t}<br>Context: {c}"
                    for t, c in zip(df_day["tokens"], df_day["context"])
                ],
                hoverinfo="text",
            )
        )

    # Add traces for shapes (with/without space)
    for marker, label in [("circle", "No Space"), ("cross", "With Space")]:
        df_marker = pca_df[pca_df["marker"] == marker]
        fig.add_trace(
            go.Scatter(
                x=df_marker["PC2"],
                y=df_marker["PC3"],
                mode="markers",
                marker=dict(symbol=marker, size=8, color="rgba(0,0,0,0)"),
                name=label,
                legendgroup="shapes",
                legendgrouptitle_text="Token Type",
                text=[
                    f"Token: {t}<br>Context: {c}"
                    for t, c in zip(df_marker["tokens"], df_marker["context"])
                ],
                hoverinfo="text",
            )
        )

    # Update layout
    fig.update_layout(
        height=800,
        width=800,
        title_text=f"PCA Analysis - Cluster {fs_splitting_cluster} (Months)",
        xaxis_title="PC2",
        yaxis_title="PC3",
        legend=dict(groupclick="toggleitem", tracegroupgap=20),
    )

    # Update traces with thinner marker borders
    fig.update_traces(
        marker=dict(
            size=12,
            line=dict(
                width=0.5,  # Reduced from 2 to 0.5
                color="DarkSlateGrey",
            ),
        ),
        selector=dict(mode="markers"),
    )

    outer_suffix = "" if not plot_inner else "_inner"

    if save_figs:
        # Save as PNG
        png_path = os.path.join(
            pca_path, f"pca_plot_months_{fs_splitting_cluster}{outer_suffix}.png"
        )
        fig.write_image(png_path, scale=3.0)

        # Save as HTML
        html_path = os.path.join(
            pca_path, f"pca_plot_months_{fs_splitting_cluster}{outer_suffix}.html"
        )
        fig.write_html(html_path)
    else:
        fig.show()

    return fig

In [28]:
plot_pca_months(pca_df, pca_path, fs_splitting_cluster, save_figs=True)

In [29]:
plot_pca_months(pca_df, pca_path, fs_splitting_cluster, plot_inner=True, save_figs=True)

In [30]:
# plot_subgraph_static(
#     load_subgraph(results_path, activation_threshold, subgraph_id),
#     node_df,
#     0.0,
#     os.path.join(pca_path, "overall_subgraph"),
#     overall_feature_activations,
#     normalize_globally=False,
#     save_figs=True,
# )

FileNotFoundError: Subgraph file not found: /Users/matthew/Documents/Github/sae_cooccurrence/results/gpt2-small/res-jb/blocks_0_hook_resid_pre/n_batches_1000/subgraph_objects/activation_1_5/subgraph_2644.pkl

In [31]:
plot_subgraph_from_sparse_matrix_path(
    results_path,
    pca_path,
    fs_splitting_cluster,
    node_df,
    save_figs=True,
    matrix_filename="sparse_thresholded_matrix_1_5.npz",
)

/Users/matthew/Documents/Github/sae_cooccurrence/sae_cooccurrence/pca.py:3322: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



(<networkx.classes.graph.Graph at 0x3ee487c50>,
       node_id  feature_activations
 2941    17827                  397
 2942     3877                  583
 2943    15526                  517
 2944     2662                  498
 2945    17351                  351
 2946     6890                  419
 2947     4622                  351
 2948    15727                  457
 2949    18994                  407
 2950    10676                  215
 2951     8789                  433
 2952    22458                  510)